# Assignment 12 - Natural Language Processing

##### 1. Load the tokenized Paradise Lost from the Gutenberg Corpus in NLTK.

- Stem or lemmatize the words and find counts.

In [1]:
!pip3 install nltk

import 

Defaulting to user installation because normal site-packages is not writeable


- Select the top 20 words and create a bar chart.

- Exclude stop words and make sure you are including words of all capitalizations in your count. If there are any meaningless “words” (“thus” and single letters, etc.) that are produced in your list of top words, alter your logic to exclude them.

- Specify why you chose stemming or lemmatization.

#### 2. Perform Vader Sentiment Analysis on the book.

- Find the 5 most negative, 5 most positive, and 5 most neutral sentences in Paradise Lost. This may take a while to run, so you can always start with a small subset of the data (100 sentences) and then once your code works as expected, expand it to the whole book and let it run.

#### 3. Explain your findings from the previous question. Are the sentences and their sentiment analysis scores correct? Explain why or why not.